In [3]:
#finds site map of entire domain
from usp.tree import sitemap_tree_for_homepage

#for saving and loading objects
import pickle

#for accessing url html
import requests
import urllib.request, urllib.parse, urllib.error
from urllib.request import Request, urlopen

#for html parsing
from bs4 import BeautifulSoup

#anxillary functions/methods
import numpy as np
import pandas as pd
import time
import random
import datetime
from dateutil.tz import tzutc, tzoffset
from tqdm.notebook import tqdm

#tree = sitemap_tree_for_homepage('https://www.scmp.com');


In [12]:
all_urls = []

for page in tree.all_pages():
    all_urls.append(page.url)

urls = []
#only opinion pieces
for url in all_urls:
    if "https://www.scmp.com/comment" in url:
        if "https://www.scmp.com/comment/letters" not in url:
            urls.append(url)
        
pickle.dump(urls, open("SCMP_Opinion_URLS.p", "wb" ) )
print(len(urls))


14178


In [5]:
!ls ../Pickles

SCMP_All_Articles.pkl SCMP_Opinion_URLS.p
SCMP_Broken_Opinion.p SCMP_Opinions.pkl


In [6]:
urls = pickle.load(open('../Pickles/SCMP_Opinion_URLS.p', 'rb'))
print(len(urls))

author = []
title = []
link = []
summary = []
date = []
broken = []

14178


In [4]:
#using BS4 to request each html data
for i in tqdm(urls):
    try:
        
        req = Request(str(i), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')

        by = soup.find("meta", {"name": "cse_author"})
        t = soup.find("meta", {"name": "cse_headline"})
        d = soup.find("meta", {"property": "article:published_time"})
        summ = soup.find("meta", {"name": "cse_summary"})


        if by and t and d and summ:
            author.append(by["content"])
            title.append(t["content"])
            date.append(d["content"])
            summary.append(summ["content"])
            link.append(i)

            if len(link)% 1000 == 0:
                print("Dataset contains " + str(len(link)) + " entries.")
    except:
        print("URL number " + str(urls.index(i)) + " is broken: " + i )
        broken.append(i)
        pass

URL number 149 is broken: https://www.scmp.com/comment/opinion/article/3092560/when-coronavirus-and-racism-collide-good-bad-and-ugly
URL number 553 is broken: https://www.scmp.com/comment/opinion/article/3084609/ocean-park-not-too-big-fail-if-anything-it-has-failed-hong-kong-and
URL number 605 is broken: https://www.scmp.com/comment/opinion/article/3083590/misguided-hong-kong-rebels-still-deserve-second-chance
URL number 607 is broken: https://www.scmp.com/comment/insight-opinion/article/2148737/pro-beijing-lawmakers-do-right-thing-foreign-judges
URL number 608 is broken: https://www.scmp.com/comment/insight-opinion/article/2148736/extra-effort-needed-stop-hong-kong-wasting-water
URL number 611 is broken: https://www.scmp.com/comment/insight-opinion/united-states/article/2187216/china-has-right-its-own-economic-vision-us
URL number 40 is broken: https://www.scmp.com/comment/opinion/article/3094628/hong-kong-elections-shameful-absence-women-politics
URL number 382 is broken: https://www

In [4]:
pickle.dump(broken, open("SCMP_Broken_Opinion.p", "wb" ) )

df = pd.DataFrame(list(zip(title, author, date, summary, link)), 
               columns =['Title', 'Author', 'Date', 'Summary','Link']) 

df.to_pickle("./SCMP_Opinions.pkl")


NameError: name 'broken' is not defined

# Data Analysis

In [4]:
df = pd.read_pickle(r'../Pickles/SCMP_Opinions.pkl')
SCMPEditorial = df[df['Author'] == 'SCMP Editorial']
SCMPEditorial = SCMPEditorial.reset_index(drop=True)

In [8]:
df.Date

0        2020-07-31T05:21:58+08:00
1        2020-07-31T05:15:48+08:00
2        2020-07-31T04:00:48+08:00
3        2020-07-30T00:44:25+08:00
4        2020-07-30T00:20:19+08:00
                   ...            
14094    2020-05-09T22:40:26+08:00
14095    2020-05-09T04:43:56+08:00
14096    2020-05-09T04:12:05+08:00
14097    2020-05-08T21:06:33+08:00
14098    2020-05-11T06:30:22+08:00
Name: Date, Length: 14099, dtype: object